# 09 - 종합 실습

## (1) 환경 준비

### 1) 라이브러리 불러오기

In [2]:
# 판다스, 넘파이 라이브러리 불러오기
import pandas as pd
import numpy as np

### 2) 데이터 불러오기

다음 세개의 데이터를 불러와서 저장합시다.

* sales : https://raw.githubusercontent.com/DA4BAM/dataset/master/sales.csv
* products : https://raw.githubusercontent.com/DA4BAM/dataset/master/products.csv
* customers : https://raw.githubusercontent.com/DA4BAM/dataset/master/customers2.csv

In [3]:
sales = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/sales.csv')
products = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/products.csv')
customers = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/customers2.csv')

세 데이터프레임의 상위 5개 행을 조회하시오.

In [55]:
sales.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70012 entries, 0 to 70011
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   OrderID     70012 non-null  int64 
 1   Seq         70012 non-null  int64 
 2   OrderDate   70012 non-null  object
 3   ProductID   70012 non-null  object
 4   Qty         70012 non-null  int64 
 5   Amt         70012 non-null  int64 
 6   CustomerID  70012 non-null  object
dtypes: int64(4), object(3)
memory usage: 3.7+ MB


In [56]:
products.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ProductID    62 non-null     object
 1   ProductName  62 non-null     object
 2   Category     62 non-null     object
 3   SubCategory  62 non-null     object
 4   CategoryOrd  62 non-null     int64 
dtypes: int64(1), object(4)
memory usage: 2.5+ KB


In [20]:
customers.head()

,CustomerID,RegisterDate,Gender,BirthYear
0,c328222,2014-09-25,F,1960
1,c281448,2013-06-18,F,1974
2,c038336,2003-10-10,F,1968
3,c084237,2007-03-09,F,1982
4,c162600,2010-06-14,F,1978


## (2) 데이터 집계, 수정, 조회하기

### 1) sales의 ProductID 별 판매량(Qty)과 판매액(Amt)의 합계를 tmp에 저장하고 조회하시오.

In [10]:
tmp = sales.groupby('ProductID')[['Qty', 'Amt']].sum()
tmp

,Qty,Amt
ProductID,,
p1001771,1055,3354827
p1002841,903,11011541
p1005621,906,2601703
p1005771,3963,7319963
p1005891,5194,10119037
...,...,...
p1246581,2180,2456346
p1255281,809,955633
p1256521,701,1241545


### 2)  1번 결과를 판매액 기준 내림차순으로 정렬하고 상위 5개 상품을 조회하시오.  
(매출액 상위 5개 상품을 조회하시오.)

In [11]:
tmp.sort_values('Amt', ascending = False).head()

,Qty,Amt
ProductID,,
p1072601,4058,18129067
p1178011,1653,14078818
p1002841,903,11011541
p1005891,5194,10119037
p1194801,990,7517664


### 3) customers의 성별(Gender) 별 고객 수를 조회하시오.

In [18]:
customers['Gender'].value_counts()

F    2060
M     183
Name: Gender, dtype: int64

### 4) customers의 출생연도(BirthYear) 로 부터 나이(Age)를 계산해서 변수로 추가하시오.

In [11]:
customers['Age']= 2022-customers['BirthYear']

### 5) customers의 나이(Age)를 나이대(AgeGroup)로 변환하시오.

* [ , 30)  : 30미만
* [30, 40) : 30대
* [40, 50) : 40대
* [50, 60) : 50대
* [60, 70) : 60대
* [70, )   : 70이상

> [30, 40) 의 의미 : 30 <= ,  < 40

In [12]:
bin = [0,30,40,50,60,70,100]
customers['AgeGroup'] = pd.cut(customers['Age'], bins = bin, right = False, labels = ['30대미만', '30대','40대', '50대', '60대', '70이상' ])

### 6) customers의 나이대(AgeGroup) 별 고객수를 구해 봅시다.

In [13]:
customers.groupby('AgeGroup').count()

,CustomerID,RegisterDate,Gender,BirthYear,Age
AgeGroup,,,,,
30대미만,6,6,6,6,6
30대,186,186,186,186,186
40대,925,925,925,925,925
50대,749,749,749,749,749
60대,267,267,267,267,267
70이상,110,110,110,110,110


### 7) 매출액 top 5 상품명을 조회해 봅시다.

* 매출액은 sales에 있고, 상품명은 products에 있습니다. 한꺼번에 조회하려면 merge를 사용해야 합니다.
    * 1) sales와 products를 ProductID 기준으로 inner 조인을 합시다.
    * 2) 상품명 별 매출액 합계를 집계합니다. 
    * 3) 매출액 합계 기준 내림차순으로 정렬하고 top 5를 조죄합니다.


In [15]:
# 1) sales와 products를 ProductID 기준으로 inner 조인을 합시다.
tmp = pd.merge(sales, products, on = 'ProductID', how = 'inner')


In [16]:
# 2) 2) 상품명 별 매출액 합계를 집계합니다.
tmp.groupby('ProductName')['Amt'].sum()


ProductName
감귤컵푸딩        677154
감자칩         3676518
고구마스틱       1620986
깻잎          2930359
낱개캔디         479197
느타리버섯       4163035
당근          4453387
두부_대       10119037
두부_소        7319963
딸기_대       11011541
딸기_소        7517664
딸기아이스크림      940786
딸기요거트        955633
마늘          3354827
무           1260905
무농약시금치      2469392
배아이스크림       623612
백오이         2036463
부추          1070348
부침용두부       1241545
브로컬리        3491762
뿌셔뿌셔        1215673
사과_대        5399285
사과_소       14078818
사과아이스크림      794018
상추          1494844
새우깡         2340237
소보루빵         536294
숙주나물        2601703
순두부          820101
시금치         2665640
쌈모음         3229161
애호박         3421170
양배출         1478820
양파           762702
연두부         1017937
열무          1867542
우유1000     18129067
우유200       2359250
유기농우유       6437323
유부          2113187
저지방우유       4404124
짱구          2161256
참외          5491030
청오이         3960918
초코아이스크림     1158483
초코우유        2456346
초코콘          374057
콘칩           942184
콩나물     

In [17]:
# 3) 매출액 합계 기준 내림차순으로 정렬하고 top 5를 조죄합니다.

tmp.groupby('ProductName')[['Amt']].sum().sort_values('Amt', ascending = False).head()

,Amt
ProductName,
우유1000,18129067
사과_소,14078818
딸기_대,11011541
두부_대,10119037
딸기_소,7517664


### 8) 연령대 별 매출액을 조회합시다.

* 매출액은 sales에 있고, 연령대(AgeGroup)은 customers에 있습니다. 한꺼번에 조회하려면 merge를 사용해야 합니다.


In [18]:
# 1) sales와 customers를  CustomerID 기준으로 inner 조인을 합시다.

tmp = pd.merge(sales, customers, on = 'CustomerID', how = 'inner')
tmp

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,RegisterDate,Gender,BirthYear,Age,AgeGroup
0,107,2,2016-01-02,p1036481,2,2100,c150417,2010-03-03,F,1974,48,40대
1,107,1,2016-01-02,p1175481,1,1300,c150417,2010-03-03,F,1974,48,40대
2,185,1,2016-01-04,p1162631,1,4600,c150417,2010-03-03,F,1974,48,40대
3,67,2,2016-01-11,p1012751,1,1350,c150417,2010-03-03,F,1974,48,40대
4,201,3,2016-01-12,p1005891,1,1950,c150417,2010-03-03,F,1974,48,40대
...,...,...,...,...,...,...,...,...,...,...,...,...
70007,193,5,2017-03-31,p1179241,2,4400,c017735,2002-07-31,F,1960,62,60대
70008,193,6,2017-03-31,p1144661,1,2500,c017735,2002-07-31,F,1960,62,60대
70009,193,7,2017-03-31,p1012581,1,1050,c017735,2002-07-31,F,1960,62,60대
70010,193,9,2017-03-31,p1052661,1,1545,c017735,2002-07-31,F,1960,62,60대


In [19]:
# 2) AgeGroup 별 매출액 합계를 집계합니다.

tmp.groupby('AgeGroup')[['Amt']].sum()

,Amt
AgeGroup,
30대미만,167539
30대,11730152
40대,86042546
50대,70047957
60대,19286478
70이상,7724796


### 9) [심화]연령대 별, 상품 카테고리 매출 비중(%)을 조회합시다.

* 1) 세 데이터프레임을 merge 해야 합니다.
    * merge는 한번에 두개씩만 가능하므로, 두번 수행합니다.
* 2) for loop를 이용하여, 각 연령대에 대해서 다음의 작업을 반복 수행합니다. 
    * 하나의 연령대 값에 대해서, 
        * 상품 카테고리(Category)별 매출액 합계를 구합니다.
        * 연령대에 해당하는 전체 매출액으로 나눠서, 카테고리별 비율을 계산합니다.
        * 결과를 출력합니다.



In [21]:
# 1) 세 데이터프레임을 merge 
tmp = pd.merge(products, sales, on = 'ProductID', how = 'inner')
tmp = pd.merge(tmp, customers, on = 'CustomerID', how = 'inner')
tmp

,ProductID,ProductName,Category,SubCategory,CategoryOrd,OrderID,Seq,OrderDate,Qty,Amt,CustomerID,RegisterDate,Gender,BirthYear,Age,AgeGroup
0,p1052661,새우깡,간식,과자,3,83,7,2016-01-02,1,1455,c085558,2007-04-05,F,1974,48,40대
1,p1052661,새우깡,간식,과자,3,139,17,2016-07-13,1,1318,c085558,2007-04-05,F,1974,48,40대
2,p1052661,새우깡,간식,과자,3,171,3,2016-10-12,1,1455,c085558,2007-04-05,F,1974,48,40대
3,p1052661,새우깡,간식,과자,3,177,23,2017-02-21,1,1545,c085558,2007-04-05,F,1974,48,40대
4,p1052661,새우깡,간식,과자,3,199,12,2017-03-08,1,1545,c085558,2007-04-05,F,1974,48,40대
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70007,p1144661,양배출,채소,잎채소,5,3,1,2016-09-28,1,1650,c261447,2012-11-02,F,1976,46,40대
70008,p1144661,양배출,채소,잎채소,5,43,1,2016-12-22,1,1950,c142648,2009-11-10,F,1967,55,50대
70009,p1144661,양배출,채소,잎채소,5,149,4,2017-03-23,1,2500,c266623,2013-01-09,F,1981,41,40대
70010,p1211951,무농약시금치,채소,잎채소,5,45,1,2016-09-14,-5,-19250,c330348,2014-10-16,F,1969,53,50대


In [24]:
ag = tmp['AgeGroup'].unique().sort_values()
ag

['30대미만', '30대', '40대', '50대', '60대', '70이상']
Categories (6, object): ['30대미만' < '30대' < '40대' < '50대' < '60대' < '70이상']

In [22]:
# 2) for loop를 이용하여, 각 연령대에 대해서 다음의 작업을 반복 수행합니다

# 하나의 연령대 값에 대해서,
# 상품 카테고리(Category)별 매출액 합계를 구합니다.
# 연령대에 해당하는 전체 매출액으로 나눠서, 카테고리별 비율을 계산합니다.
# 결과를 출력합니다.

ag = tmp['AgeGroup'].unique().sort_values()

for a in ag :
    tmp1 = tmp.loc[tmp['AgeGroup'] == a]
    t_amt = tmp1['Amt'].sum()
    tmp2 = tmp1.groupby('Category')['Amt'].sum() / t_amt
    print(tmp2)

# result.append(((tmp.loc[tmp['AgeGroup'] == '30대미만'].groupby('Category')[['Amt']].sum() / tmp.loc[tmp['AgeGroup'] == '30대미만'][['Amt']].sum() ) * 100).round(2))
# print(tmp.loc[tmp['AgeGroup'] == '30대'].groupby('Category')[['Amt']].sum())
# result.append(((tmp.loc[tmp['AgeGroup'] == '30대'].groupby('Category')[['Amt']].sum() / tmp.loc[tmp['AgeGroup'] == '30대'][['Amt']].sum() ) * 100).round(2))

Category
간식     0.080877
과일     0.088935
반찬류    0.310787
유제품    0.134631
채소     0.384770
Name: Amt, dtype: float64
Category
간식     0.068632
과일     0.264618
반찬류    0.129206
유제품    0.255305
채소     0.282240
Name: Amt, dtype: float64
Category
간식     0.083996
과일     0.257663
반찬류    0.138119
유제품    0.257045
채소     0.263177
Name: Amt, dtype: float64
Category
간식     0.057381
과일     0.256595
반찬류    0.171246
유제품    0.229995
채소     0.284783
Name: Amt, dtype: float64
Category
간식     0.034329
과일     0.226639
반찬류    0.242396
유제품    0.164646
채소     0.331990
Name: Amt, dtype: float64
Category
간식     0.025016
과일     0.279025
반찬류    0.269608
유제품    0.108956
채소     0.317394
Name: Amt, dtype: float64


In [36]:
result

[            Amt
 Category       
 간식         8.05
 과일         6.34
 반찬류       23.25
 유제품       14.74
 채소        47.62,
             Amt
 Category       
 간식         6.95
 과일        24.44
 반찬류       13.15
 유제품       29.30
 채소        26.16]